In [13]:
import pandas as pd
import csv
import numpy as np
import math
import time
import matplotlib.pyplot as plt

import sklearn
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cross_validation import train_test_split
from sklearn import svm
from sklearn.svm import SVC
from sklearn import linear_model
from sklearn import tree
from sklearn.cross_validation import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report
from sklearn.svm import LinearSVC


from keras.models import Sequential

from pprint import pprint

In [15]:
NCAA_Seasons = pd.read_csv('NCAA_Final.csv')

NCAA_Seasons.head()



,team_id,season,team_name,team_mascot,seed,total_wins,total_losses,total_games_played,adj_win_perc,str_of_sched,...,average_field_goals_att,average_three_points_made,average_three_points_att,average_free_throws_made,average_free_throws_att,average_two_points_made,average_two_points_att,average_points_in_paint,average_efficiency,average_true_shooting_att
0,9b166a3f-e64b-4825-bb6b-92c6f0418263,2017,Arizona,Wildcats,S4,27,8,35,0.7707,0.5515,...,57.23,6.77,18.34,16.09,21.17,22.06,38.89,33.60,93.97,66.54
1,77a69fb0-1355-4342-ac09-b4cc7949d95e,2017,Murray State,Racers,E12,24,6,32,0.8056,0.4869,...,56.34,8.47,22.47,15.75,21.47,18.72,33.88,32.88,90.63,65.79
2,61a3908a-7492-4b6f-809e-12c61976bb0a,2017,Loyola (IL),Ramblers,S11,31,6,38,0.8596,0.5352,...,51.66,7.29,18.42,11.95,16.47,18.89,33.24,32.95,83.11,58.91
3,6ed15092-2670-450a-99c2-61d861e87644,2017,Texas,Longhorns,S10,19,15,34,0.5274,0.5699,...,59.12,7.21,22.53,12.97,19.50,18.74,36.59,32.82,76.74,67.70
4,b2fda957-e15c-4fb2-8a13-6e58496f561e,2017,Oklahoma,Sooners,M10,18,14,32,0.5484,0.5597,...,64.44,9.38,26.13,15.94,21.38,20.44,38.31,36.94,93.44,73.84


In [17]:
pprint(NCAA_Seasons.columns)

Index(['team_id', 'season', 'team_name', 'team_mascot', 'seed', 'total_wins',
       'total_losses', 'total_games_played', 'adj_win_perc', 'str_of_sched',
       'rpi', 'total_minutes_played', 'total_field_goals_made',
       'total_field_goals_att', 'total_field_goals_pct',
       'total_two_points_made', 'total_two_points_att', 'total_two_points_pct',
       'total_three_points_made', 'total_three_points_att',
       'total_three_points_pct', 'total_blocked_att', 'total_free_throws_made',
       'total_free_throws_att', 'total_free_throws_pct',
       'total_offensive_rebounds', 'total_defensive_rebounds',
       'total_rebounds', 'total_assists', 'total_turnovers',
       'total_assists_turnover_ratio', 'total_steals', 'total_blocks',
       'total_personal_fouls', 'total_tech_fouls', 'total_points',
       'total_fast_break_pts', 'total_flagrant_fouls',
       'total_points_off_turnovers', 'total_second_chance_pts',
       'total_ejections', 'total_foulouts', 'total_points_in_paint

In [18]:
E_2017 = ["Villanova", "Butler", "St. Bonaventure", "Murray State", "Marshall", "Stephen F. Austin", "Cal State Fullerton", 
         "Radford", "Purdue", "Texas Tech", "Wichita State", "West Virginia", "Florida", "Arkansas", "Virginia Tech", "Alabama"]

S_2017 = ["Virginia", "Texas", "Loyola (IL)", "Davidson", "Buffalo", "Wright State", "Georgia State", "UMBC", "Cincinnati", "Tennessee", 
          "Arizona", "Kentucky", "Miami (FL)", "Nevada", "Creighton", "Kansas State"]

M_2017 = ["Kansas", "Oklahoma", "Syracuse", "New Mexico State", "Charleston", "Bucknell", "Iona", "Pennsylvania", "Duke", 
          "Michigan State", "Auburn", "Clemson", "TCU", "Rhode Island", "Seton Hall", "North Carolina State"]

W_2017 = ["Xavier", "Providence", "San Diego State", "South Dakota State", "UNC Greensboro", "Montana", "Lipscomb", "Texas Southern", 
          "North Carolina", "Michigan", "Gonzaga", "Ohio State", "Houston", "Texas A&M", "Missouri", "Florida State"]

E_2018 = ["Villanova", "Butler", "St. Bonaventure", "Murray State", "Marshall", "Stephen F. Austin", "Cal State Fullerton", 
         "Radford", "Purdue", "Texas Tech", "Wichita State", "West Virginia", "Florida", "Arkansas", "Virginia Tech", "Alabama"]

S_2018 = ["Virginia", "Texas", "Loyola (IL)", "Davidson", "Buffalo", "Wright State", "Georgia State", "UMBC", "Cincinnati", "Tennessee", 
          "Arizona", "Kentucky", "Miami (FL)", "Nevada", "Creighton", "Kansas State"]

M_2018 = ["Kansas", "Oklahoma", "Syracuse", "New Mexico State", "Charleston", "Bucknell", "Iona", "Pennsylvania", "Duke", 
          "Michigan State", "Auburn", "Clemson", "TCU", "Rhode Island", "Seton Hall", "North Carolina State"]

W_2018 = ["Xavier", "Providence", "San Diego State", "South Dakota State", "UNC Greensboro", "Montana", "Lipscomb", "Texas Southern", 
          "North Carolina", "Michigan", "Gonzaga", "Ohio State", "Houston", "Texas A&M", "Missouri", "Florida State"]

In [6]:
def getSeasonData(team_market, season):
    year_data = NCAA_Seasons[NCAA_Seasons['season'] == year]
    
    return [adj_win_perc, str_of_sched, rpi, total_field_goals_pct, 
            total_two_points_pct, total_three_points_pct, total_free_throws_pct, total_true_shooting_pct,
            average_rebounds, average_blocks, average_turnovers, average_efficiency, average_fast_break_pts,
            average_points_off_turnovers, average_second_chance_pts, average_minutes, average_points]
    

In [14]:
def compareTwoTeams(team_market_1, team_market_2, season):
    team_1 = getSeasonData(team_market, year)
    team_2 = getSeasonData(team_market, year)
    diff = [a - b for a, b in zip(team_1, team_2)]
    return diff

In [15]:
kansas_id = NCAA_Seasons[NCAA_Seasons['team_name'] == 'Kansas'].values[0][0]
compareTwoTeams(kansas_id, kentucky_id, 2017)

NameError: name 'NCAA_Seasons' is not defined